In [1]:
import os
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup

import pandas as pd

In [2]:
os.environ['WDM_LOG_LEVEL'] = '0'

# 
s = Service(ChromeDriverManager().install())
chrome_options = Options()
chrome_options.add_argument('--headless')
chrome_options.add_argument('window-size=1920x1080')
chrome_options.add_argument('--disable-gpu')

driver = webdriver.Chrome(service=s, chrome_options=chrome_options)

driver.set_page_load_timeout(100)

url = "http://www3.saude.ba.gov.br/cgi/deftohtm.exe?sinan/deng.def"
# url = "http://www3.saude.ba.gov.br/cgi/deftohtm.exe?sinan/chikun.def"
# url = "http://www3.saude.ba.gov.br/cgi/deftohtm.exe?sinan/zika.def"
driver.get(url)

WebDriverWait(driver, 1).until(EC.presence_of_element_located((By.XPATH, '/html/body/div/center/div/form/div[4]/div[2]/div[2]/input[1]')))

# Linha
driver.find_element(By.XPATH, '/html/body/div/center/div/form/div[2]/div/div[1]/select/option[1]').click()
# Coluna
driver.find_element(By.XPATH, '/html/body/div/center/div/form/div[2]/div/div[2]/select/option[5]').click()
# Periodo
driver.find_element(By.XPATH, '/html/body/div/center/div/form/div[3]/div/select/option[2]').click()
driver.find_element(By.XPATH, '/html/body/div/center/div/form/div[3]/div/select/option[1]').click()

# Mostra
driver.find_element(By.XPATH, '/html/body/div/center/div/form/div[4]/div[2]/div[2]/input[1]').click()


WebDriverWait(driver, 1).until(EC.presence_of_element_located((By.XPATH, '/html/body/div/div[3]/table/tbody/tr/td[1]/a')))

html = driver.page_source

driver.quit()

/tmp/ipykernel_36128/2402893610.py:10: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(service=s, chrome_options=chrome_options)


In [3]:
soup = BeautifulSoup(html, 'html.parser')

tabdados = soup.select(".tabdados tbody tr td ")

tabdados = list(map(lambda node: node.get_text().strip(), tabdados))

col_tabdados = soup.select(".tabdados th ")
col_tabdados = list(map(lambda node: node.get_text().strip(), col_tabdados))

In [4]:
tam_lin = int(len(tabdados)/len(col_tabdados))
tam_col = len(col_tabdados)

aux = []
for i in range (tam_lin):
    aux_2 = []
    for j in range (tam_col):  
        aux_2.append(tabdados.pop(0))
    aux.append(aux_2)

df = pd.DataFrame(aux, columns=col_tabdados)

In [5]:
df = df.drop(columns=['Total'])
df = df.drop(0)
df = df.reset_index()
df = df.drop(columns=['index'])

In [6]:
aux = list(df.iloc[:, 0])
for i in range (len(aux)):
    aux[i] = aux[i].split(" ")
    while (len(aux[i]) > 2):
        aux[i][1] = aux[i][1] + " " + aux[i][2]
        aux[i].pop(2)
aux = pd.DataFrame(aux, columns=["Codigo", "Município"])
df.iloc[:, 0] = aux["Município"]

In [7]:
colunas = [ df["Jan"], df["Fev"], df["Mar"], df["Abr"], df["Mai"], df["Jun"], df["Jul"], df["Ago"], df["Set"], df["Out"], df["Nov"], df["Dez"] ]

for i in range (len(colunas)):
    for j in range (len(colunas[i])):
        try:
            colunas[i][j] = int(colunas[i][j])
        except:
            colunas[i][j] = 0

In [10]:
df = pd.DataFrame(list(zip(df.iloc[:, 0], colunas[0], colunas[1], colunas[2], colunas[3], colunas[4], colunas[5], colunas[6], colunas[7], colunas[8], colunas[9], colunas[10], colunas[11])), columns = ['Municipio', 'Jan', 'Fev', 'Mar', 'Abr', 'Mai', 'Jun', 'Jul', 'Ago',
       'Set', 'Out', 'Nov', 'Dez'])

In [11]:
df.to_csv("../scr/mes_municipio.csv", encoding='utf-8')

In [12]:
df

,Municipio,Jan,Fev,Mar,Abr,Mai,Jun,Jul,Ago,Set,Out,Nov,Dez
0,Abaíra,0,0,0,0,0,0,0,1,0,0,0,0
1,Abaré,0,0,0,1,0,0,0,0,0,0,1,0
2,Acajutiba,0,0,0,0,0,1,0,1,0,0,0,1
3,Adustina,5,0,0,0,3,0,0,0,1,0,0,0
4,Água Fria,0,0,0,0,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
345,Vitória da Conquista,87,70,157,102,83,48,35,35,38,32,33,47
346,Wagner,0,0,2,0,1,0,3,0,0,0,0,1
347,Wanderley,0,1,0,0,0,0,0,0,0,0,0,0
348,Wenceslau Guimarães,0,0,0,0,0,1,1,0,0,0,0,0
